# PubChemLite visualizations
Here the PubChemLite (PCL) visualizations will be made using UMAP. 

In [ ]:
import pandas as pd

import pickle

In [ ]:
#pcl dataset with labels
with open('2025-04-29_pcl_visualization_dataframe.pkl', 'rb') as f:
    pcl = pickle.load(f)

#pcl dataset with SIRIUS fingerprints
pcl_umap_coord = pd.read_csv('PubChemLite_20250415_SIRIUS5_positiveFP_UMAP_coordinates.tsv', sep='\t')

with open('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Ellinor - Master thesis/Data/Tox21/2025-03-12_Tox21_remade_fp.pkl', 'rb') as f:
    tox21 = pickle.load(f)

In [ ]:
pcl.head()

In [ ]:
pcl_umap_coord.head()

In [ ]:
tox21['tox21'] = True

tox21

### Cleaning of coordinate SMILES

In [ ]:
import rdkit
from rdkit import Chem, rdBase, RDLogger, DataStructs
RDLogger.DisableLog('rdApp.*')
from rdkit.Chem import AllChem, Draw, inchi, rdDepictor, PandasTools, SaltRemover, Descriptors
from rdkit.Chem.Draw import IPythonConsole, rdMolDraw2D
from rdkit.Chem.MolStandardize import rdMolStandardize

In [ ]:
PandasTools.AddMoleculeColumnToFrame(pcl_umap_coord, smilesCol='SMILES_pug', molCol='ROMol')

In [ ]:
pcl_umap_coord_nona = pcl_umap_coord[~pcl_umap_coord.ROMol.isna()]

Two compounds needed to be removed, as they had a disallowed SMILES description. When searching for them in PubChem, it was described that these two compounds () had a disallowed valence conformation according to MMFF94.

In [ ]:
uncharger = rdMolStandardize.Uncharger()  # neutralize the molecule (if possible)
pcl_umap_coord_nona['ROMol'] = pcl_umap_coord_nona.ROMol.apply(lambda x: uncharger.uncharge(x))

pcl_umap_coord_nona.ROMol.apply(lambda x: Chem.RemoveStereochemistry(x)) #Remove stereochemistry
pcl_umap_coord_nona['SMILES'] = pcl_umap_coord_nona.ROMol.apply(lambda x: Chem.MolToSmiles(x)) #Update SMILES
pcl_umap_coord_nona['InChIKey'] = pcl_umap_coord_nona.ROMol.apply(lambda x: inchi.MolToInchiKey(x)) #Generate InChIKey
pcl_umap_coord_nona['InChIKey14'] = pcl_umap_coord_nona.InChIKey.apply(lambda x: x.split('-')[0]) #Generate InChIKey14

In [ ]:
pcl_merged = pd.merge(pcl, pcl_umap_coord_nona[['InChIKey14', 'x_axis', 'y_axis']], on='InChIKey14', how='inner')

In [ ]:
pcl_merged

In [ ]:
pcl_merged = pd.merge(pcl_merged, tox21[['InChIKey14', 'tox21', 'nr.ahr', 'sr.mmp']], on='InChIKey14', how='left')

In [ ]:
pcl_merged[pcl_merged.tox21.notna()]

In [ ]:
# with open('2025-05-07_pcl_vis_final.pkl', 'wb') as f:
#     pickle.dump(pcl_merged, f)

### Visualizations

In [ ]:
with open('2025-05-07_pcl_vis_final.pkl', 'rb') as f:
    pcl_merged = pickle.load(f)

In [ ]:
pcl_merged

In [ ]:
pcl_merged.gcms_spectra.sum(), pcl_merged.lcms_spectra.sum()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.axes as axes
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties

import matplotlib
import seaborn as sns

In [ ]:
# Tox21 data
pcl_tox21 = pcl_merged[pcl_merged.tox21.notna()]
pcl_no_tox21 = pcl_merged[pcl_merged.tox21.isna()]

pcl_tox21_ahr_active = pcl_merged[pcl_merged['nr.ahr']==1]    #ahr active
pcl_tox21_ahr_inactive = pcl_merged[pcl_merged['nr.ahr']==0]   #ahr inactive
pcl_no_tox21_ahr = pcl_merged[pcl_merged['nr.ahr'].isna()|(pcl_merged['nr.ahr']==-999999)]

pcl_tox21_mmp_active = pcl_merged[pcl_merged['sr.mmp']==1]    #mmp active
pcl_tox21_mmp_inactive = pcl_merged[pcl_merged['sr.mmp']==0]    #mmp inactive
pcl_no_tox21_ahr = pcl_merged[pcl_merged['sr.mmp'].isna()|(pcl_merged['sr.mmp']==-999999)]

# GC/LC availability data
gc_true = pcl_merged[pcl_merged['gcms_spectra'] == True]
gc_false = pcl_merged[pcl_merged['gcms_spectra'] == False]

lc_true = pcl_merged[pcl_merged['lcms_spectra'] == True]
lc_false = pcl_merged[pcl_merged['lcms_spectra'] == False]

#LoGP data
pcl_merged['XLogP_pubchem'] = pcl_merged['XLogP_pubchem'].astype(float)

xlogp = pcl_merged[pcl_merged['XLogP_pubchem'].notna()]
no_xlogp = pcl_merged[pcl_merged['XLogP_pubchem'].isna()]
print(f'Describe xlogp:\n{xlogp['XLogP_pubchem'].describe()}')

#BP data
bp = pcl_merged[pcl_merged['boiling_point_matched_cleaned'].notna()]
not_bp = pcl_merged[pcl_merged['boiling_point_matched_cleaned'].isna()]
print(f'Describe bp:\n{bp['boiling_point_matched_cleaned'].describe()}')

In [ ]:
print(f'XLogP\n90% qunatile:\n {xlogp["XLogP_pubchem"].quantile(0.9)}\n10% qunatile:\n {xlogp["XLogP_pubchem"].quantile(0.1)}')
print(f'Boiling point\n90% qunatile:\n {bp["boiling_point_matched_cleaned"].quantile(0.9)}\n10% qunatile:\n {bp["boiling_point_matched_cleaned"].quantile(0.1)}')

#### Add all plots into one plot

In [ ]:
#PCL original plot
def pcl_plot(ax):
    ax.scatter(pcl_merged.x_axis, 
                pcl_merged.y_axis,
                alpha=0.2,
                s=0.7,
                c='#023047'
                )

    ax.set_title('Original PubChemLite UMAP projection', fontweight='normal',
                 fontsize=14)

    ax.set_aspect('equal', adjustable='box')

In [ ]:
# GC/LC availability plots
def gc_plot(ax):
    ax.scatter(gc_false.x_axis, 
                gc_false.y_axis, 
                alpha=0.2, 
                s=0.7,
                label = 'GCMS-spectra not available',
                c='#023047'
                )

    ax.scatter(gc_true.x_axis, 
                gc_true.y_axis, 
                alpha=0.2, 
                s=0.7,
                label = 'GCMS-spectra available',
                c='#8ECAE6'
                )

    ax.set_title('D.', loc='left', fontweight='bold',
                 fontsize=14)

    legend_handles = [
        mlines.Line2D([], [], color='#8ECAE6', marker='o', linestyle='None', markersize=5, label='GCMS-spectra available', alpha=1)
    ]

    ax.legend(handles=legend_handles, 
            loc='upper right')

    ax.set_aspect('equal', adjustable='box')

def lc_plot(ax):
    ax.scatter(lc_false.x_axis, 
                lc_false.y_axis, 
                alpha=0.2, 
                s=0.7,
                c='#023047'
                )

    ax.scatter(lc_true.x_axis, 
                lc_true.y_axis, 
                alpha=0.5, 
                s=0.7,
                c='#219EBC'
                )

    ax.set_title('E.', loc='left', fontweight='bold',
                 fontsize=14)

    legend_handles = [
        mlines.Line2D([], [], color='#219EBC', marker='o', linestyle='None', markersize=5, label='LCMS-spectra available', alpha=1)
    ]

    title_font = FontProperties()
    title_font.set_weight('bold')

    ax.legend(handles=legend_handles, 
            loc='upper right')

    ax.set_aspect('equal', adjustable='box') 

In [ ]:
# Make custom gradient for colorbar
from matplotlib.colors import LinearSegmentedColormap

colors = ['#023047','#219EBC', '#8ECAE6', '#FFB703', '#FB8500']

custom_cmap = LinearSegmentedColormap.from_list("my_colormap", colors, N=256)

# Use it in a plot
plt.imshow([[0,1]], cmap=custom_cmap)
plt.colorbar()
plt.title("Custom Color Gradient")
plt.axis('off')
plt.show()

In [ ]:
# XLogP and BP plots
def xlogp_plot(ax):

    # Grey points (without XLogP values)
    ax.scatter(
        no_xlogp.x_axis,
        no_xlogp.y_axis,
        color='grey',
        alpha=0.2,
        label='No XLogP',
        s=0.2
    )

        # Color-coded points (with XLogP values)
    scatter_with_xlogp = ax.scatter(
        xlogp.x_axis,
        xlogp.y_axis,
        c=xlogp.XLogP_pubchem,
        cmap=custom_cmap,
        vmin=0,
        vmax=10,
        alpha=1,
        s = 0.5
    )

    ax.set_title('F.', loc='left', fontweight='bold',
                 fontsize=14)

    # Add colorbar
    #ax.colorbar(scatter_with_xlogp, label='XLogP (PubChem)')

    legend_handles = [
        mlines.Line2D([], [], color='grey', marker='o', linestyle='None', markersize=5, label='No available data', alpha=1)
    ]

    ax.legend(handles=legend_handles, 
            loc='upper right')

    ax.set_aspect('equal', adjustable='box')

def bp_plot(ax):
    # Grey points (without XLogP values)
    ax.scatter(
        not_bp.x_axis,
        not_bp.y_axis,
        color='grey',
        alpha=0.1,
        label='No XLogP',
        s=0.2
    )

    # Color-coded points (with XLogP values)
    scatter_with_bp = ax.scatter(
        bp.x_axis,
        bp.y_axis,
        c=bp.boiling_point_matched_cleaned,
        cmap=custom_cmap,
        vmax=366.4,
        vmin=92.94,
        alpha=1,
        s = 0.7
    )

    # Labels and title
    ax.set_title('G.',loc='left', fontweight='bold',
                 fontsize=14)

    # Add colorbar
    #plt.colorbar(scatter_with_bp, label='Boiling point (PubChem)')

    legend_handles = [
        mlines.Line2D([], [], color='grey', marker='o', linestyle='None', markersize=5, label='No available data', alpha=1)
    ]

    title_font = FontProperties()
    title_font.set_weight('bold')

    ax.legend(handles=legend_handles, 
            loc='upper right',
            title_fontproperties=title_font)

    ax.set_aspect('equal', adjustable='box')


In [ ]:
color_dict = {'active':'#FB8500',
              'inactive':'#219EBC'}

In [ ]:

# Tox21 plots 

def tox21_plot(ax):
    ax.scatter(pcl_no_tox21.x_axis,
                pcl_no_tox21.y_axis,
                alpha=0.2,
                s = 0.7,
                c='#023047'
                )

    # Grey points (without XLogP values)
    ax.scatter( pcl_tox21.x_axis,
                pcl_tox21.y_axis,
                alpha=0.5,
                s=0.7,
                c='#FB8500'
            )

    ax.set_title('A.', loc='left', fontweight='bold',
                 fontsize=14)


    legend_handles = [
        mlines.Line2D([], [], color='#FB8500', marker='o', linestyle='None', markersize=5, label='Tox21', alpha=1)
    ]

    ax.legend(handles=legend_handles, 
            loc='upper right')

    ax.set_aspect('equal', adjustable='box')

def tox21_AHR_plot(ax):
    # Grey points (without XLogP values)
    ax.scatter(
        pcl_no_tox21_ahr.x_axis,
        pcl_no_tox21_ahr.y_axis,
        color='#023047',
        alpha=0.1,
        s=0.2
    )

    #scatter with inactives
    ax.scatter(
        pcl_tox21_ahr_inactive.x_axis,
        pcl_tox21_ahr_inactive.y_axis,
        c=color_dict['inactive'],
        label=['AhR inactive'],
        alpha=0.8,
        s = 0.5
    )
       #Scatter with actives
    ax.scatter(
        pcl_tox21_ahr_active.x_axis,
        pcl_tox21_ahr_active.y_axis,
        c=color_dict['active'],
        label=['AhR active'],
        alpha=1,
        s = 0.5
    )

    # Labels and title
    ax.set_title('B.',loc='left', fontweight='bold',
                 fontsize=14)
    
    title_font = FontProperties()
    title_font.set_weight('bold')

    legend_handles = [
        mlines.Line2D([], [], color=color_dict['active'], marker='o', linestyle='None', markersize=5, label='AhR active', alpha=1),
        mlines.Line2D([], [], color=color_dict['inactive'], marker='o', linestyle='None', markersize=5, label='AhR inactive', alpha=1)
    ]

    ax.legend(handles=legend_handles, 
            loc='upper right')

    ax.set_aspect('equal', adjustable='box')

def tox21_MMP_plot(ax):
    # Grey points (without XLogP values)
    ax.scatter(
        pcl_merged.x_axis,
        pcl_merged.y_axis,
        color='#023047',
        alpha=0.1,
        s=0.2
    )

    #scatter with inactives
    ax.scatter(
        pcl_tox21_mmp_inactive.x_axis,
        pcl_tox21_mmp_inactive.y_axis,
        c=color_dict['inactive'],
        alpha=0.8,
        s = 0.5
    )
       #Scatter with actives
    ax.scatter(
        pcl_tox21_mmp_active.x_axis,
        pcl_tox21_mmp_active.y_axis,
        c=color_dict['active'],
        alpha=1,
        s = 0.5
    )

    # Labels and title
    ax.set_title('C.',loc='left', fontweight='bold',
                 fontsize=14)
    
    title_font = FontProperties()
    title_font.set_weight('bold')
    
    legend_handles = [
        mlines.Line2D([], [], color=color_dict['active'], marker='o', linestyle='None', markersize=5, label='MMP active', alpha=1),
        mlines.Line2D([], [], color=color_dict['inactive'], marker='o', linestyle='None', markersize=5, label='MMP inactive', alpha=1)
    ]

    ax.legend(handles=legend_handles, 
            loc='upper right')

    ax.set_aspect('equal', adjustable='box')

In [ ]:
# Set the figure parameters
plt.rcParams.update({'figure.figsize':[9.8,9.8],
                'font.size': 12, 
                'axes.titlesize': 12,
                'axes.labelsize': 12,
                'xtick.labelsize': 12,
                'ytick.labelsize': 12,
                'legend.fontsize': 12,
                'legend.title_fontsize': 12,
                'axes.titleweight': 'bold',
                'font.family': 'serif',
                'font.serif': ['Times New Roman'],
                'figure.dpi':300,
                
                })

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axes = plt.subplots(4, 2, figsize=(9, 15),gridspec_kw={"width_ratios": [1, 1]})  # 3 row, 2 columns
axes = axes.flatten()

pcl_plot(axes[0])
axes[0].tick_params(axis='both',  which='both',left=False, right=False, labelleft=False, labelright=False)
axes[0].xaxis.set_ticks([])
tox21_plot(axes[1])
axes[1].tick_params(axis='both', which='both', left=False, right=False, labelleft=False, labelright=False)
axes[1].xaxis.set_ticks([])
tox21_AHR_plot(axes[2])
axes[2].tick_params(axis='both', which='both', left=False, right=False, labelleft=False, labelright=False)
axes[2].xaxis.set_ticks([])
tox21_MMP_plot(axes[3])
axes[3].tick_params(axis='both', which='both', left=False, right=False, labelleft=False, labelright=False)
axes[3].xaxis.set_ticks([])
gc_plot(axes[4])
axes[4].tick_params(axis='both', which='both', left=False, right=False, labelleft=False, labelright=False)
axes[4].xaxis.set_ticks([])
lc_plot(axes[5])
axes[5].tick_params(axis='both', which='both', left=False, right=False, labelleft=False, labelright=False)
axes[5].xaxis.set_ticks([])
xlogp_plot(axes[6])
axes[6].tick_params(axis='both', which='both', left=False, right=False, labelleft=False, labelright=False)
axes[6].xaxis.set_ticks([])
bp_plot(axes[7])
axes[7].tick_params(axis='both', which='both', left=False, right=False, labelleft=False, labelright=False)
axes[7].xaxis.set_ticks([])


# Add colorbars to the last two plots
divider = make_axes_locatable(axes[6])
cax_logp = divider.append_axes("right", size="3%", pad=0.05)

divider = make_axes_locatable(axes[7])
cax_bp = divider.append_axes("right", size="3%", pad=0.05)

sc_xlogp = axes[6].collections[1]  # get the first scatter (with color)
cbar_logp = fig.colorbar(
    sc_xlogp, 
    ax=axes[6], 
    cax=cax_logp,
    extend='both')
cbar_logp.set_label("XLogP, A.U.", rotation=270, labelpad=8)

sc_bp = axes[7].collections[1]  # get the second scatter (with color)
cbar_bp = fig.colorbar(
    sc_bp, 
    ax=axes[7], 
    cax=cax_bp,
    extend='both')
cbar_bp.set_label("Boiling point, °C", rotation=270, labelpad=15)

# Set the title for the entire figure
fig.supxlabel('', fontsize=12, y=0.07)
fig.supylabel('', fontsize=12,x=0.08)

#fig.tight_layout(rect=[0, 0, 0, 0])  # Adjust layout to make room for the title

#fig.tight_layout()
#plt.show()

fig.savefig("2025-05-23_pubchemlite_visualization.png", dpi=300, bbox_inches='tight', transparent=True)

### Boiling point box plot

In [ ]:
with open('2025-05-07_pcl_vis_final.pkl', 'rb') as f:
    pcl_merged = pickle.load(f)

In [ ]:
pcl_merged

In [ ]:
ahr_bp = pcl_merged[(pcl_merged['nr.ahr'].notna()) & (pcl_merged['nr.ahr']!= -999999)]
ahr_bp = ahr_bp[['boiling_point_matched_cleaned', 'nr.ahr']]
ahr_bp['labels'] = ahr_bp['nr.ahr'].apply(lambda x: 'Inactive' if x == 0 else 'Active')

In [ ]:
ahr_bp

In [ ]:
mmp_bp = pcl_merged[(pcl_merged['sr.mmp'].notna()) & (pcl_merged['sr.mmp']!= -999999)]
mmp_bp = mmp_bp[['boiling_point_matched_cleaned', 'sr.mmp']]
mmp_bp['labels'] = mmp_bp['sr.mmp'].apply(lambda x: 'Inactive' if x == 0 else 'Active')
mmp_bp

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt


fig, ax = plt.subplots(1, 2, figsize=(12, 5), sharey=True, sharex=True)  # 1 row, 2 columns

# Create the boxplot
sns.boxplot(x='labels', y='boiling_point_matched_cleaned', data=ahr_bp, ax=ax[0], palette=['#219EBC', '#FB8500'])
ax[0].set_title('AhR', fontweight='bold', fontsize=14)
ax[0].set_ylabel('Boiling Point, °C')
ax[0].set_xlabel('')

sns.boxplot(x='labels', y='boiling_point_matched_cleaned', data=mmp_bp, ax=ax[1], palette=['#219EBC', '#FB8500'])
ax[1].set_title('MMP', fontweight='bold', fontsize=14)
ax[1].set_xlabel('')

plt.subplots_adjust(wspace=0.1) 


# Customize labels and title

plt.savefig('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Visualizations/2025-05-22_boxplot_endpoint_bp.png', dpi=300, bbox_inches='tight', transparent=True)

### Histograms

In [ ]:
ahr_bp_active = ahr_bp[ahr_bp['nr.ahr'] == 1]
ahr_bp_inactive = ahr_bp[ahr_bp['nr.ahr'] == 0]

mmp_bp_active = mmp_bp[mmp_bp['sr.mmp'] == 1]
mmp_bp_inactive = mmp_bp[mmp_bp['sr.mmp'] == 0]

In [ ]:
len(ahr_bp_active), len(ahr_bp_inactive), len(mmp_bp_active), len(mmp_bp_inactive)

In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(10, 5), sharey=True, sharex=True)  # 1 row, 2 columns

sns.histplot(data=ahr_bp_active, x='boiling_point_matched_cleaned', label='Active', color='#FB8500', edgecolor=None, kde=False, stat='density', bins=50, ax=ax[0])
sns.histplot(data=ahr_bp_inactive, x='boiling_point_matched_cleaned', label='Inactive', color='#219EBC', edgecolor=None, kde=False, stat='density', bins=60, ax=ax[0])
ax[0].set_title('AhR', fontweight='bold', fontsize=14)

ax[0].set_xlabel('')
ax[0].set_ylabel('')

ax[0].tick_params(axis='y', left=False, labelleft=False)

sns.histplot(data=mmp_bp_active, x='boiling_point_matched_cleaned', label='Active', color='#FB8500', edgecolor=None,  kde=False, stat='density', bins=50, ax=ax[1])
sns.histplot(data=mmp_bp_inactive, x='boiling_point_matched_cleaned', label='Inactive', color='#219EBC', edgecolor=None,  kde=False, stat='density', bins=50, ax=ax[1])
ax[1].set_title('MMP', fontweight='bold', fontsize=14)

ax[1].set_xlabel('')
ax[1].set_ylabel('')

ax[1].tick_params(axis='y', left=False, labelleft=False)

# Add legend and labels
ax[0].legend()
ax[1].legend()
plt.tight_layout()
fig.supxlabel('Boiling Point, °C', fontsize=12, y=0.0)

plt.savefig('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Visualizations/2025-05-22_histogram_endpoint_bp_comparison.png', dpi=300, bbox_inches='tight', transparent=True)

### LogP histogram

In [ ]:
import pickle
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
with open('2025-05-07_pcl_vis_final.pkl', 'rb') as f:
    pcl_merged = pickle.load(f)

In [ ]:
pcl_merged.head()

In [ ]:
ahr_logp = pcl_merged[(pcl_merged['nr.ahr'].notna()) & (pcl_merged['nr.ahr']!= -999999)]
ahr_logp = ahr_logp[['XLogP_pubchem', 'nr.ahr']]
ahr_logp['labels'] = ahr_logp['nr.ahr'].apply(lambda x: 'Inactive' if x == 0 else 'Active')
ahr_logp

In [ ]:
mmp_logp = pcl_merged[(pcl_merged['sr.mmp'].notna()) & (pcl_merged['sr.mmp']!= -999999)]
mmp_logp = mmp_logp[['XLogP_pubchem', 'sr.mmp']]
mmp_logp['labels'] = mmp_logp['sr.mmp'].apply(lambda x: 'Inactive' if x == 0 else 'Active')
mmp_logp

In [ ]:
ahr_logp_active = ahr_logp[ahr_logp['nr.ahr'] == 1]
ahr_logp_inactive = ahr_logp[ahr_logp['nr.ahr'] == 0]

mmp_logp_active = mmp_logp[mmp_logp['sr.mmp'] == 1]
mmp_logp_inactive = mmp_logp[mmp_logp['sr.mmp'] == 0]

In [ ]:

fig, ax = plt.subplots(1, 2, figsize=(10, 5), sharey=True, sharex=True)  # 1 row, 2 columns

sns.histplot(data=ahr_logp_active, x='XLogP_pubchem', label='Active', color='#FB8500', edgecolor=None, kde=False, stat='density', bins=30, ax=ax[0])
sns.histplot(data=ahr_logp_inactive, x='XLogP_pubchem', label='Inactive', color='#219EBC', edgecolor=None, kde=False, stat='density', bins=80, ax=ax[0])
ax[0].set_title('AhR', fontweight='bold', fontsize=14)

ax[0].set_xlabel('')
ax[0].set_ylabel('')

ax[0].tick_params(axis='y', left=False, labelleft=False)

sns.histplot(data=mmp_logp_active, x='XLogP_pubchem', label='Active', color='#FB8500', edgecolor=None,  kde=False, stat='density', bins=30, ax=ax[1])
sns.histplot(data=mmp_logp_inactive, x='XLogP_pubchem', label='Inactive', color='#219EBC', edgecolor=None,  kde=False, stat='density', bins=80, ax=ax[1])
ax[1].set_title('MMP', fontweight='bold', fontsize=14)

ax[1].set_xlabel('')
ax[1].set_ylabel('')

ax[1].tick_params(axis='y', left=False, labelleft=False)

# Add legend and labels
ax[0].legend()
ax[1].legend()
plt.tight_layout()
fig.supxlabel('XLogP, A.U.', fontsize=12, y=0.0)

plt.savefig('/Users/elli/Library/CloudStorage/OneDrive-Kruvelab/Master_thesis/Visualizations/2025-05-23_histogram_endpoint_logp_comparison.png', dpi=300, bbox_inches='tight', transparent=True)